In [1]:
import sys, os
%matplotlib notebook
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import torch
import torch.nn.functional as F
from torch.autograd import Variable, grad
import torch.distributions as distrib
import torch.nn as nn
import torch.utils.data as td
from torch.utils.data import Dataset, DataLoader
import gym
import numpy as np

import foundation as fd
import torch.multiprocessing as mp
import time
import pickle

import foundation.models.networks as nets
from foundation.lattice.core import Expansion, Node
from foundation import util

In [4]:
x = torch.arange(1, 17).reshape(1, 1 ,4, 4).float()
y = torch.nn.functional.unfold(x, 3, padding=1)
z = torch.nn.functional.fold(y, 4, 3, padding=1)
print(z / x)

tensor([[[[ 4.,  6.,  6.,  4.],
          [ 6.,  9.,  9.,  6.],
          [ 6.,  9.,  9.,  6.],
          [ 4.,  6.,  6.,  4.]]]])


In [5]:
x.size(), y.size(), z.size()

(torch.Size([1, 1, 4, 4]), torch.Size([1, 9, 16]), torch.Size([1, 1, 4, 4]))

In [7]:
x = torch.arange(1, 17).reshape(1, 1 ,4, 4).float()
x /= x
y = torch.nn.functional.unfold(x, 3, padding=1)
z = torch.nn.functional.fold(y, 4, 3, padding=1)

In [8]:
z

tensor([[[[ 4.,  6.,  6.,  4.],
          [ 6.,  9.,  9.,  6.],
          [ 6.,  9.,  9.,  6.],
          [ 4.,  6.,  6.,  4.]]]])

tensor([[[[ 1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.]]]])

In [44]:
B = 1
H, W = 4, 4
k = 3
w = torch.randn(1, k**2, H*W)
x = torch.randn(B, 1, H, W)
w /= w
w.size(), x.size()

(torch.Size([1, 9, 16]), torch.Size([1, 1, 4, 4]))

In [45]:
unfold = nn.Unfold(k, padding=(k-1)//2)
fold = nn.Fold((H,W), k, padding=(k-1)//2)
ux = unfold(x)
ux.size()

torch.Size([1, 9, 16])

In [46]:
uxw = ux * w
uxw.size()

torch.Size([1, 9, 16])

In [47]:
uxw[0,0]

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.8071, -1.1410,
        -1.2309,  0.0000,  0.7088, -0.5855, -0.2398,  0.0000,  1.0789,
        -0.7219, -2.3161])

In [48]:
slow = nn.Fold((H,W), (1,1))(uxw.sum(1, keepdim=True))
slow

tensor([[[[ 0.7895, -0.6812, -3.5993, -1.8729],
          [ 1.1466, -2.6402, -8.1678, -5.7195],
          [ 0.8343, -3.0663, -7.1173, -6.0683],
          [ 0.7109, -2.9499, -6.0653, -5.6017]]]])

In [56]:
fast = fold(uxw)
fast # not right - sums the terms that map back to the original pixel

tensor([[[[  7.2284,  -6.8459,  -7.3852,  -0.7016],
          [  4.2531,  -5.2691,  -2.1582,  -1.3609],
          [  6.4736,  -6.4970, -20.8449,  -9.1830],
          [  0.3824,   1.5497,  -8.0683,  -1.6414]]]])

In [50]:
simple = uxw.sum(1, keepdim=True).view(B, 1, H, W)
simple # sums weighted nn

tensor([[[[ 0.7895, -0.6812, -3.5993, -1.8729],
          [ 1.1466, -2.6402, -8.1678, -5.7195],
          [ 0.8343, -3.0663, -7.1173, -6.0683],
          [ 0.7109, -2.9499, -6.0653, -5.6017]]]])

In [51]:
x

tensor([[[[ 1.8071, -1.1410, -1.2309, -0.1754],
          [ 0.7088, -0.5855, -0.2398, -0.2268],
          [ 1.0789, -0.7219, -2.3161, -1.5305],
          [ 0.0956,  0.2583, -1.3447, -0.4103]]]])

In [4]:
n11 = Node()

RuntimeError: dimension out of range (expected to be in range of [-1, 0], but got 1)